# Processing Site Lidar

In this notebook we preprocess out lidar sites. This includes:
- Reprojection
- Cropping to the site geometry
- (Optional) classifying ground points
- Computing height above ground
- Remove statistical outliers (noise)
- Saving cloud as a cloud optimised point cloud (COPC)

To process the data we will use PDAL pipelines.
We will also use dask to run the processing in parallel.

In [10]:
from pathlib import Path
import json

from jinja2 import Template
import geopandas as gpd
import pdal
import pandas as pd

## PDAL Pipeline Template

Each run of PDAL processing is defined by a pipeline represented by a JSON string. Our processing for each site is mostly the same. Only a few site varaibles will change each run (e.g. name of input file, polygon to clip to). To replace variables we'll use the Jinja template engine. Variables are denoted by `{{ variable_name }}`

In [11]:
# Note, we could have this just a string, but as a dict allows us to add comments
pipeline_template_dict = [
    # Read the input LAS file
    {
        "type": "readers.las",
        "filename": "{{ input_path }}"
    },
    # Reproject to MGA2020 + Aus Height Datum
    {
        "type": "filters.reprojection",
        "out_srs": "EPSG:7855+5711"
    },
    # Crop to our site polygon
    # This is optional, but useful as our source sites are quite large
    # and we don't need the whole thing
    {
        "type": "filters.crop",
        "polygon": "{{ polygon_wkt }}"
    },
    # Note, if you wanted to calculate your own ground classification points
    # do so here. We'll keep the ground classificaiton provided by VirtualTas.
    #
    # Calculate height above ground
    {
        "type": "filters.hag_nn"
    },
    # Remove any noise points
    {
        "type": "filters.outlier",
        "method": "statistical",
        "mean_k": 6,
        "multiplier": 10
    },
    # Save as a COPC file
    {
        "type": "writers.copc",
        "filename": "{{ output_path }}",
        "forward": "scale,offset",
        "extra_dims": "all"
    }
]

pipeline_template = json.dumps(pipeline_template_dict, indent=2)

# Function to replace variables
def replace_pipeline_variables(pipeline_template: str, context: dict):
    template = Template(pipeline_template)
    return template.render(context)

### Site pipelines

We use some data from our previously created `sites.geojson` to create the pdal pipelines.

In [12]:
import geopandas as gpd

sites_gdf = gpd.read_file("../data/outputs/sites/sites.geojson")
sites_gdf.head()

,site,id,geometry
0,AGG_O_01,AGG_O_01,"POLYGON ((463303.79 5259716.755, 463123.329 52..."
1,AGG_O_05,AGG_O_05,"POLYGON ((455430.465 5284117.991, 455191.829 5..."
2,AGG_O_07,AGG_O_07,"POLYGON ((464747.381 5299156.759, 464706.037 5..."
3,AGG_Y_02,AGG_Y_02,"POLYGON ((491855.984 5230960.243, 491825.641 5..."
4,AGG_Y_03,AGG_Y_03,"POLYGON ((490748.752 5208804.286, 490664.016 5..."


There are 3 variables in the pipeline above:
- input_path - Where to source the lidar file for that site
- output_path - Where to save the processed lidar file
- polygon_wkt - The polygon for that site in well known text (WKT) format

In [13]:
data_dir = Path("../data")
lidar_source_dir = data_dir / "source" / "cycle-2"  # cycle-2 has best coverage
lidar_output_dir = data_dir / "outputs" / "sites" / "lidar"
lidar_output_dir.mkdir(parents=True, exist_ok=True)

def create_pipeline_from_site(site_row):
    site_id = site_row['id']

    context = {
        "input_path": str(lidar_source_dir / f"{site_id}.laz"),
        "output_path": str(lidar_output_dir / f"{site_id}.copc.laz"),
        "polygon_wkt": site_row.geometry.wkt
    }
    return replace_pipeline_variables(pipeline_template, context)

pipelines = sites_gdf.apply(create_pipeline_from_site, axis=1).to_list()
print(pipelines[0])


[
  {
    "type": "readers.las",
    "filename": "../data/source/cycle-2/AGG_O_01.laz"
  },
  {
    "type": "filters.reprojection",
    "out_srs": "EPSG:7855+5711"
  },
  {
    "type": "filters.crop",
    "polygon": "POLYGON ((463303.78970947 5259716.755331744, 463123.3287796102 5259746.770813609, 463046.07269726234 5259761.7233045045, 462950.04695010127 5259805.900973975, 462980.706530192 5259890.517699114, 463221.4219030455 5259835.555207977, 463327.6009013069 5259803.548345104, 463303.78970947 5259716.755331744))"
  },
  {
    "type": "filters.hag_nn"
  },
  {
    "type": "filters.outlier",
    "method": "statistical",
    "mean_k": 6,
    "multiplier": 10
  },
  {
    "type": "writers.copc",
    "filename": "../data/outputs/sites/lidar/AGG_O_01.copc.laz",
    "forward": "scale,offset",
    "extra_dims": "all"
  }
]


## Processing Pipelines

PDAL is built around processing these pipelines.

In [14]:
def process_pdal_pipeline(pipeline: str, return_data: bool = False):
    """
    Process a PDAL pipeline string.

    Args:
        pipeline (str): The PDAL pipeline JSON string.
        return_data (bool): If True, return the PDAL Pipeline object after execution. Defaults to False. Returning pipeline data
        will contain metadata and all the points processed by the pipeline. This can be a large object so defaults to False.
    """
    pipeline_obj = pdal.Pipeline(pipeline)
    count = pipeline_obj.execute()  # Execute the pipeline
    return (count, pipeline_obj if return_data else None)

Processing a single pipeline can take some time.

In [ ]:
%%time

(count, pl) = process_pdal_pipeline(pipelines[0], return_data=True)
print(f"Processed {count} points.")

points = pl.arrays[0]
points_df = pd.DataFrame(pl.arrays[0])
points_df.head()

Processed 2500328 points.
CPU times: user 20.4 s, sys: 238 ms, total: 20.7 s
Wall time: 18.4 s


,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,Synthetic,...,ScanAngleRank,UserData,PointSourceId,GpsTime,ScanChannel,Red,Green,Blue,Infrared,HeightAboveGround
0,475982.849,5230972.054,405.449,27275,4,4,1,0,2,0,...,25.152000,3,2,4.145390e+08,0,18761,19789,16705,27184,0.000
1,475982.974,5230972.033,414.119,28035,3,4,1,0,0,0,...,25.337999,47,2,4.145390e+08,0,27242,25957,18247,28649,8.670
2,475982.282,5230971.713,417.752,30093,1,2,1,0,0,0,...,25.434000,65,2,4.145390e+08,0,11822,13878,15163,13407,12.116
3,475982.753,5230971.415,413.309,27449,3,4,1,0,0,0,...,25.242001,42,2,4.145390e+08,0,32639,31097,22873,30545,7.673
4,475982.043,5230971.425,412.609,29700,3,4,1,0,0,0,...,25.337999,38,2,4.145390e+08,0,27756,26985,18761,29291,6.973


### Parallel processing with Dask

In [36]:
from dask.distributed import Client

client = Client()  # Start a Dask client
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54370,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:54383,Total threads: 2
Dashboard: http://127.0.0.1:54386/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:54373,


In [40]:
futures = client.map(process_pdal_pipeline, pipelines, key=sites_gdf['id'].tolist())

In [46]:
# TODO - better error manangement and reporting
print(futures[0].result())

(3572396, None)
